In [1]:
import re, os, time
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
from gensim.models.ldamodel import LdaModel
from gensim.models import Phrases
from gensim.models.phrases import Phraser
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Import nltk stopwords and spacy for lemmatization
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
import spacy
%matplotlib inline

# Enable logging for Gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings('ignore',category=DeprecationWarning)

import time
import pickle
import bs4

# Collect names of Tweet files
from os import listdir
from os.path import isfile, join
import json

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
import textacy
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from bokeh.charts import Chord

from pysankey import sankey

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\ProgramData\Anaconda3\lib\site-packages\bokeh\util\deprecation.py:34: BokehDeprecationWarning: 
The bokeh.charts API has moved to a separate 'bkcharts' package.

This compatibility shim will remain until Bokeh 1.0 is released.
After that, if you want to use this API you will have to install
the bkcharts package explicitly.

  warn(message)


In [2]:
# Print one topic
# Load a potentially pretrained model
# from gensim.test.utils import datapath
# temp_file = datapath('lda_multicore_full') 
lda_model = LdaModel.load('lda_model_full')
all_topic = []
for i in range(lda_model.num_topics):
    all_topic += [lda_model.show_topic(i)]
# all_topic

In [3]:
all_topic_words = []
all_topic_num = []

for i in range(len(all_topic)):
    one_topic_words = []
    one_topic_num = []
    for j in range(len(all_topic[i])):
        one_topic_words += [all_topic[i][j][0]]
        one_topic_num += [all_topic[i][j][1]]
        
    all_topic_words += [one_topic_words]
    all_topic_num += [one_topic_num]
    

In [178]:
all_topic_group_num = []
for topic in range (len(all_topic)):
    all_group = []
    count = 0

    while count < len(all_topic[topic]):
        single_group = all_topic[topic][count]

        range_a = abs(all_topic_num[topic][count] - np.array(all_topic_num[topic][count:len(all_topic_num[topic])]).std())

        for j in range(count,len(all_topic[topic])-1):  

            if count == (len(all_topic[topic])-1):
                if all_topic_num[topic][-1] <= range_a:
                    single_group = (single_group) + (all_topic[topic][-1])              

            elif count < (len(all_topic[topic])-1):       
                if all_topic_num[topic][count+1] >= range_a:
                    single_group = (single_group) + (all_topic[topic][count+1])
                    count += 1    

        all_group += [single_group]
        count += 1
        
    all_topic_group_num += [all_group]

In [179]:
all_topic_group_num[0]

[('vvd', 0.032720536, 'nederland', 0.029464105),
 ('pvv', 0.020109527, 'partij', 0.018944632),
 ('rutte', 0.015990036, 'mee', 0.015465685, 'doet', 0.014456638),
 ('sigrid_kaag', 0.013964481, 'gewoon', 0.012887054),
 ('lid', 0.011083391)]

In [176]:
all_topic_group_words = []
for topic in range (len(all_topic)):
    all_group = []
    count = 0

    while count < len(all_topic[topic]):
        single_group = (all_topic_words[topic][count],)

        range_a = abs(all_topic_num[topic][count] - np.array(all_topic_num[topic][count:len(all_topic_num[topic])]).std())

        for j in range(count,len(all_topic[topic])-1):  

            if count == (len(all_topic[topic])-1):
                if all_topic_num[topic][-1] <= range_a:
                    single_group = tuple(list(single_group) + ([all_topic_words[topic][-1]]))            

            elif count < (len(all_topic[topic])-1):       
                if all_topic_num[topic][count+1] >= range_a:
                    single_group = tuple(list(single_group) + ([all_topic_words[topic][count+1]])) 
                    count += 1    

        all_group += [single_group]
        count += 1
        
    all_topic_group_words += [all_group]

In [182]:
from ipysankeywidget import SankeyWidget
from ipywidgets import Layout
from IPython.display import display

In [193]:
layout = Layout(width="1000", height="200")
def sankey(margin_top=10, **value):
    """Show SankeyWidget with default values for size and margins"""
    return SankeyWidget(layout=layout,
                        margins=dict(top=margin_top, bottom=0, left=100, right=100),
                        **value)

In [197]:
all_topic_group_words[0]

[('vvd', 'nederland'),
 ('pvv', 'partij'),
 ('rutte', 'mee', 'doet'),
 ('sigrid_kaag', 'gewoon'),
 ('lid',)]

In [194]:
links = [
    {'source': 'vvd', 'target': 'pvv', 'value': 3, 'color': 'steelblue'},
    {'source': 'vvd', 'target': 'partij', 'value': 3, 'color': 'steelblue'},
    {'source': 'nederland', 'target': 'pvv', 'value': 3, 'color': 'red'},
    {'source': 'nederland', 'target': 'partij', 'value': 3, 'color': 'red'},
    {'source': 'pvv', 'target': 'rutte', 'value': 2, 'color': 'olive'},
    {'source': 'pvv', 'target': 'mee', 'value': 2, 'color': 'olive'},
    {'source': 'pvv', 'target': 'doet', 'value': 2, 'color': 'olive'},
    {'source': 'partij', 'target': 'rutte', 'value': 2, 'color': 'khaki'},
    {'source': 'partij', 'target': 'mee', 'value': 2, 'color': 'khaki'},
    {'source': 'partij', 'target': 'doet', 'value': 2, 'color': 'khaki'},
    {'source': 'rutte', 'target': 'sigrid_kaag', 'value': 2, 'color': 'aqua'},
    {'source': 'rutte', 'target': 'gewoon', 'value': 2, 'color': 'aqua'},
    {'source': 'mee', 'target': 'sigrid_kaag', 'value': 2, 'color': 'plum'},
    {'source': 'mee', 'target': 'gewoon', 'value': 2, 'color': 'plum'},
    {'source': 'doet', 'target': 'sigrid_kaag', 'value': 2, 'color': 'magenta'},
    {'source': 'doet', 'target': 'gewoon', 'value': 2, 'color': 'magenta'},
    {'source': 'sigrid_kaag', 'target': 'lid', 'value': 6, 'color': 'pink'},
    {'source': 'gewoon', 'target': 'lid', 'value': 6, 'color': 'pink'},
   
]
sankey(links=links)

SankeyWidget(layout=Layout(height='200', width='1000'), links=[{'source': 'vvd', 'target': 'pvv', 'value': 3, …

In [196]:
all_topic_group_words[1]

[('vvd',),
 ('rutte', 'cda'),
 ('pvda', 'gaat'),
 ('weer', 'laat', 'zijlstra'),
 ('zetels',),
 ('partij',)]

In [205]:
links = [
    {'source': 'vvd', 'target': 'rutte', 'value': 3, 'color': 'steelblue'},
    {'source': 'vvd', 'target': 'cda', 'value': 3, 'color': 'steelblue'},
    {'source': 'rutte', 'target': 'pvda', 'value': 1.5, 'color': 'red'},
    {'source': 'rutte', 'target': 'gaat', 'value': 1.5, 'color': 'red'},
    {'source': 'cda', 'target': 'pvda', 'value': 1.5, 'color': 'olive'},
    {'source': 'cda', 'target': 'gaat', 'value': 1.5, 'color': 'olive'},
    {'source': 'pvda', 'target': 'weer', 'value': 1, 'color': 'khaki'},
    {'source': 'pvda', 'target': 'laat', 'value': 1, 'color': 'khaki'},
    {'source': 'pvda', 'target': 'zijlstra', 'value': 1, 'color': 'khaki'},
    {'source': 'gaat', 'target': 'weer', 'value': 1, 'color': 'aqua'},
    {'source': 'gaat', 'target': 'laat', 'value': 1, 'color': 'aqua'},
    {'source': 'gaat', 'target': 'zijlstra', 'value': 1, 'color': 'aqua'},
    {'source': 'weer', 'target': 'zetels', 'value': 2, 'color': 'plum'},
    {'source': 'laat', 'target': 'zetels', 'value': 2, 'color': 'magenta'},
    {'source': 'zijlstra', 'target': 'zetels', 'value': 2, 'color': 'blue'},
    {'source': 'zetels', 'target': 'partij', 'value': 6, 'color': 'pink'},
    
   
]
sankey(links=links)

SankeyWidget(layout=Layout(height='200', width='1000'), links=[{'source': 'vvd', 'target': 'rutte', 'value': 3…

In [ ]:
links = [
    {'source': 'vvd', 'target': 'rutte', 'value': 6, 'color': 'steelblue'},
    {'source': 'vvd', 'target': 'cda', 'value': 6, 'color': 'steelblue'},
    {'source': 'rutte', 'target': 'pvda', 'value': 3, 'color': 'red'},
    {'source': 'rutte', 'target': 'gaat', 'value': 3, 'color': 'red'},
    {'source': 'cda', 'target': 'pvda', 'value': 3, 'color': 'olive'},
    {'source': 'cda', 'target': 'gaat', 'value': 3, 'color': 'olive'},
    {'source': 'pvda', 'target': 'weer', 'value': 2, 'color': 'khaki'},
    {'source': 'pvda', 'target': 'laat', 'value': 2, 'color': 'khaki'},
    {'source': 'pvda', 'target': 'zijlstra', 'value': 2, 'color': 'khaki'},
    {'source': 'gaat', 'target': 'weer', 'value': 2, 'color': 'aqua'},
    {'source': 'gaat', 'target': 'laat', 'value': 2, 'color': 'aqua'},
    {'source': 'gaat', 'target': 'zijlstra', 'value': 2, 'color': 'aqua'},
    {'source': 'weer', 'target': 'zetels', 'value': 4, 'color': 'plum'},
    {'source': 'laat', 'target': 'zetels', 'value': 4, 'color': 'magenta'},
    {'source': 'zijlstra', 'target': 'zetels', 'value': 4, 'color': 'blue'},
    {'source': 'zetels', 'target': 'partij', 'value': 12, 'color': 'pink'},
       
]
sankey(links=links)